Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore **regularization techniques**.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels): # flat matrix
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    # labels, 1-hot representation
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [7]:
num_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
      
    # Training computation.
    relu_layer=tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    logits = tf.matmul(relu_layer, weights_2) + biases_2
    # regularization, add small penalty on larger weights
    C = 5e-4
    R = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+C*R)
    
     # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) 
                                                          + biases_1), weights_2) + biases_2)
    test_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) 
                                                          + biases_1), weights_2) + biases_2)

In [8]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 529.311340
Minibatch accuracy: 6.2%
Validation accuracy: 25.6%
Minibatch loss at step 500: 140.488022
Minibatch accuracy: 79.7%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 99.859825
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 73.474213
Minibatch accuracy: 89.1%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 56.484444
Minibatch accuracy: 89.1%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 43.730778
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 33.928719
Minibatch accuracy: 89.1%
Validation accuracy: 84.1%
Minibatch loss at step 3500: 26.698980
Minibatch accuracy: 83.6%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 20.734749
Minibatch accuracy: 85.9%
Validation accuracy: 86.3%
Minibatch loss at step 4500: 16.133480
Minibatch accuracy: 88.3%
Validation accuracy: 85.7%
Minibatch loss at step 5000: 12.589582
Minibatch accuracy: 90.6%
Valida

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
train_labels.shape[0]

200000

In [11]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        # 用1280取代了 train_labels.shape[0], 少了很多 batch
        offset = (step * batch_size) % (1280 - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 463.892578
Minibatch accuracy: 20.3%
Validation accuracy: 18.3%
Minibatch loss at step 500: 122.236069
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 95.193604
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 74.134521
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 57.734619
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 2500: 44.961914
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 3000: 35.015800
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 3500: 27.269520
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 4000: 21.236795
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 4500: 16.539169
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 5000: 12.880699
Minibatch accuracy: 10

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
num_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
      
    # Training computation.
    # dropout with 0.5
    keep_prob = 0.5
    relu_layer=tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1),keep_prob)
    logits = tf.matmul(relu_layer, weights_2) + biases_2
    # regularization, add small penalty on larger weights
    C = 5e-4
    R = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+C*R)
    
     # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) 
                                                          + biases_1), weights_2) + biases_2)
    test_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) 
                                                          + biases_1), weights_2) + biases_2)

In [13]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        # 用1280取代了 train_labels.shape[0], 少了很多 batch
        offset = (step * batch_size) % (1280 - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 564.177246
Minibatch accuracy: 9.4%
Validation accuracy: 37.5%
Minibatch loss at step 500: 123.652725
Minibatch accuracy: 99.2%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 96.267830
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 75.149429
Minibatch accuracy: 99.2%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 58.460564
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 45.540901
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 3000: 35.473064
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 3500: 27.629719
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 4000: 21.522406
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 4500: 16.762657
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 5000: 13.055760
Minibatch accuracy: 100.0

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to **add multiple layers**.

Another one is to use **learning rate decay**:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [17]:
num_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    # learning rate decay
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(5e-2, global_step, 500, 0.96)
      
    # Training computation.
    # dropout with 0.5
    keep_prob = 0.5
    relu_layer=tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) 
                                        + biases_1),keep_prob)
    logits = tf.matmul(relu_layer, weights_2) + biases_2
    # regularization, add small penalty on larger weights
    C = 5e-4
    R = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, 
                                                                  tf_train_labels)+C*R)
    
     # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(
        learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) 
                                                          + biases_1), weights_2) + biases_2)
    test_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) 
                                                          + biases_1), weights_2) + biases_2)

In [18]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 648.647461
Minibatch accuracy: 14.1%
Validation accuracy: 18.3%
Minibatch loss at step 500: 179.138031
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Minibatch loss at step 1000: 176.229385
Minibatch accuracy: 76.6%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 160.033905
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 153.051315
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 145.062332
Minibatch accuracy: 80.5%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 142.344421
Minibatch accuracy: 78.9%
Validation accuracy: 84.0%
Minibatch loss at step 3500: 139.438431
Minibatch accuracy: 79.7%
Validation accuracy: 84.0%
Minibatch loss at step 4000: 134.352173
Minibatch accuracy: 78.9%
Validation accuracy: 83.4%
Minibatch loss at step 4500: 130.314987
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 5000: 129.913910
Minibatch accuracy: 78

https://discussions.udacity.com/t/assignment-3-problem-4-very-poor-accuracy-with-a-deeper-network/157742/18

https://discussions.udacity.com/t/assignment-3-best-model/164249

https://discussions.udacity.com/t/assignment-3-problem-4/46128

https://discussions.udacity.com/t/assignment-4-problem-2/46525/24

In [19]:
batch_size = 128
hidden_layer1_size = 1024
hidden_layer2_size = 305
hidden_lastlayer_size = 75

use_multilayers = True

regularization_meta=0.03


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    keep_prob = tf.placeholder(tf.float32)

    weights_layer1 = tf.Variable(tf.truncated_normal([image_size * image_size, 
                                                      hidden_layer1_size], stddev=0.0517))
    biases_layer1 = tf.Variable(tf.zeros([hidden_layer1_size]))

    if use_multilayers:
        weights_layer2 = tf.Variable(tf.truncated_normal([hidden_layer1_size, 
                                                          hidden_layer1_size], stddev=0.0441))
        biases_layer2 = tf.Variable(tf.zeros([hidden_layer1_size]))

        weights_layer3 = tf.Variable(tf.truncated_normal([hidden_layer1_size, 
                                                          hidden_layer2_size], stddev=0.0441))
        biases_layer3 = tf.Variable(tf.zeros([hidden_layer2_size]))
    
        weights_layer4 = tf.Variable(tf.truncated_normal([hidden_layer2_size, 
                                                          hidden_lastlayer_size], stddev=0.0809))
        biases_layer4 = tf.Variable(tf.zeros([hidden_lastlayer_size]))


    weights = tf.Variable(tf.truncated_normal([hidden_lastlayer_size 
                        if use_multilayers else hidden_layer1_size, num_labels], stddev=0.1632))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    
    # get the NN models
    def getNN4Layer(dSet, use_dropout):
        input_to_layer1 = tf.matmul(dSet, weights_layer1) + biases_layer1
        hidden_layer1_output = tf.nn.relu(input_to_layer1)
    
    
        logits_hidden1 = None
        if use_dropout:
            dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
            logits_hidden1 = tf.matmul(dropout_hidden1, weights_layer2) + biases_layer2
        else:
            logits_hidden1 = tf.matmul(hidden_layer1_output, weights_layer2) + biases_layer2
    
        hidden_layer2_output = tf.nn.relu(logits_hidden1)
    
        logits_hidden2 = None
        if use_dropout:
            dropout_hidden2 = tf.nn.dropout(hidden_layer2_output, keep_prob)
            logits_hidden2 = tf.matmul(dropout_hidden2, weights_layer3) + biases_layer3
        else:
            logits_hidden2 = tf.matmul(hidden_layer2_output, weights_layer3) + biases_layer3
    
    
        hidden_layer3_output = tf.nn.relu(logits_hidden2)
        logits_hidden3 = None
        if use_dropout:
            dropout_hidden3 = tf.nn.dropout(hidden_layer3_output, keep_prob)
            logits_hidden3 = tf.matmul(dropout_hidden3, weights_layer4) + biases_layer4
        else:
            logits_hidden3 = tf.matmul(hidden_layer3_output, weights_layer4) + biases_layer4
    
    
        hidden_layer4_output = tf.nn.relu(logits_hidden3)
        logits = None
        if use_dropout:
            dropout_hidden4 = tf.nn.dropout(hidden_layer4_output, keep_prob)
            logits = tf.matmul(dropout_hidden4, weights) + biases
        else:
            logits = tf.matmul(hidden_layer4_output, weights) + biases
    
        return logits

    # get the NN models
    def getNN1Layer(dSet, use_dropout, w1, b1, w, b):
        input_to_layer1 = tf.matmul(dSet, w1) + b1
        hidden_layer1_output = tf.nn.relu(input_to_layer1)
        
        logits = None
        if use_dropout:
            dropout_hidden1 = tf.nn.dropout(hidden_layer1_output, keep_prob)
            logits = tf.matmul(dropout_hidden1, w) + b
        else:
            logits = tf.matmul(hidden_layer1_output, w) + b
    
        return logits

  
  
    # Training computation.
    logits = getNN4Layer(tf_train_dataset, True)  
    logits_valid = getNN4Layer(tf_valid_dataset, False)
    logits_test = getNN4Layer(tf_test_dataset, False)
    
  
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss_l2 = loss + (regularization_meta * (tf.nn.l2_loss(weights)))
  
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.3, global_step, 500, 0.86, staircase=True)
  
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [21]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                     keep_prob:0.75}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
            print("Minibatch accuracy: %.1f%%" % accuracy(train_prediction.eval(
                feed_dict={tf_train_dataset : batch_data, 
                           tf_train_labels : batch_labels, keep_prob:1.0}), batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(feed_dict={keep_prob:1.0}), valid_labels))
    print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(
        feed_dict={keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0 : 2.55499
Minibatch accuracy: 33.6%
Validation accuracy: 21.3%
Minibatch loss at step 500 : 0.40131
Minibatch accuracy: 89.8%
Validation accuracy: 84.9%
Minibatch loss at step 1000 : 0.447082
Minibatch accuracy: 89.8%
Validation accuracy: 86.1%
Minibatch loss at step 1500 : 0.356933
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 2000 : 0.321453
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 2500 : 0.374637
Minibatch accuracy: 90.6%
Validation accuracy: 88.0%
Minibatch loss at step 3000 : 0.451252
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 3500 : 0.425596
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 4000 : 0.375633
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 4500 : 0.3864
Minibatch accuracy: 91.4%
Validation accuracy: 89.1%
Minibatch loss at step 5000 : 0.352223
Minibatch accuracy: 90.6%
Validation 